In [1]:
import pandas as pd
import numpy as np

In [27]:
def load_data():
    df = pd.read_csv('Match_PL.csv')
    match_ids = df['match_api_id']
    goal_dif = df['home_team_goal'].sub(df['away_team_goal'])
    goal_dif[goal_dif < 0] = -1
    goal_dif[goal_dif > 0] = 1
    home_teams = df['home_team_api_id']
    away_teams = df['away_team_api_id']

    df = pd.read_csv('Team_Attributes_PL.csv')
    columns_keep = ['team_api_id',
                    'buildUpPlaySpeed', 
                    'buildUpPlayDribbling',
                    'buildUpPlayPassing',
                    'chanceCreationPassing',
                    'chanceCreationCrossing',
                    'chanceCreationShooting',
                    'defencePressure',
                    'defenceAggression',
                    'defenceTeamWidth']
    teams_attr = df[columns_keep]
    home_columns = ['home_' + column for column in columns_keep]
    away_columns = ['away_' + column for column in columns_keep]
    columns_len = len(columns_keep) - 1
    values = np.zeros((len(match_ids), 2 * columns_len))
    for i, team in enumerate(home_teams):
        home_team_attr = teams_attr.loc[df['team_api_id'] == team].to_numpy()
        away_team_attr = teams_attr.loc[df['team_api_id'] == away_teams[i]].to_numpy()
        row = np.append(np.delete(home_team_attr, 0), np.delete(away_team_attr, 0))
        values[i,:] = row
    x = pd.DataFrame(values, index = match_ids, columns = home_columns[1:] + away_columns[1:])
    y = goal_dif
    
    tr_prop = 0.6
    cv_prop = 0.2
    te_prop = 0.2
    end_row_tr = int(tr_prop * x.shape[0])
    end_row_cv = int((tr_prop + cv_prop) * x.shape[0])
    x_tr = x[:end_row_tr]
    y_tr = y[:end_row_tr]
    x_cv = x[end_row_tr:end_row_cv]
    y_cv = y[end_row_tr:end_row_cv]
    x_te = x[end_row_cv:]
    y_te = y[end_row_cv:]
    return x_tr, y_tr, x_cv, y_cv, x_te, y_te

In [28]:
x_tr, y_tr, x_cv, y_cv, x_te, y_te = load_data()

In [29]:
print(x_tr.shape)
print(y_tr.shape)
print(x_cv.shape)
print(y_cv.shape)
print(x_te.shape)
print(y_te.shape)

(1824, 18)
(1824,)
(608, 18)
(608,)
(608, 18)
(608,)
